<a href="https://colab.research.google.com/github/tasinfrancesco/Practical_ML_PSL/blob/main/IASO_BigData2025_intro_to_SparkRDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# =========================
# Colab: PySpark classic (RDD-friendly) + remove Spark Connect conflict
# =========================

# 0) Remove the package that forces pyspark[connect] ~= 4.0.0
!pip -q uninstall -y dataproc-spark-connect google-cloud-dataproc-spark-connect || true  # package name can vary

# 1) Install Java + pinned PySpark (classic)
!apt-get update -qq
!apt-get install -y openjdk-17-jdk-headless -qq
!pip -q install pyspark==3.5.3

# 2) Configure env + start Spark
import os, subprocess
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .master("local[*]")
         .appName("colab-rdd")
         .config("spark.ui.showConsoleProgress", "false")
         .getOrCreate())

sc = spark.sparkContext
print("Spark version:", spark.version)
print(subprocess.check_output(["java","-version"], stderr=subprocess.STDOUT).decode().splitlines()[0])

# 3) Quick RDD check
rdd = sc.parallelize([1,2,3,4,5], 2).map(lambda x: (x, x*x))
print(rdd.collect())



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 11.6 MB/s eta 0:00:00
Spark version: 3.5.3
openjdk version "17.0.17" 2025-10-21
[(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]


In [16]:
!wget https://www.dropbox.com/s/7ae58iydjloajvt/shake.txt

--2025-12-17 11:38:19--  https://www.dropbox.com/s/7ae58iydjloajvt/shake.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/ao0ozilexrndwvfnce3kp/shake.txt?rlkey=udk3zb5n8ur7rj3xmqosgb7pz [following]
--2025-12-17 11:38:19--  https://www.dropbox.com/scl/fi/ao0ozilexrndwvfnce3kp/shake.txt?rlkey=udk3zb5n8ur7rj3xmqosgb7pz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6697be7bd056c6041d562fb3b6.dl.dropboxusercontent.com/cd/0/inline/C3OHNTUW8-nROV0FXFLAhmb8vaqFAzVivYsDO2ckaqKHRMT-8hgSVEiDa7-_ru1wUt_uBk-2CVnLnk5m0ym4vg2J9qwIf4p4iAoiTK5qIMFutmM-kgajYH_HAr3OI4wKvf4/file# [following]
--2025-12-17 11:38:20--  https://uc6697be7bd056c6041d562fb3b6.dl.dropboxusercontent.com/cd/0/inline/C3OHNTUW8-nROV0FXFLAhmb8vaqFAzV

In [19]:
document = sc.textFile("shake.txt.1")

In [20]:
w = document.flatMap(lambda x: x.split()).map(lambda x: (x, 1)).reduceByKey(lambda x,y: x+y)
w.collect()

[('Etext', 4),
 ('file', 14),
 ('by', 2824),
 ('Gutenberg,', 1),
 ('and', 18297),
 ('with', 6722),
 ('from', 2283),
 ('their', 1934),
 ('of', 15544),
 ('Future', 3),
 ('Shakespeare', 45),
 ('Gutenberg', 11),
 ('often', 116),
 ('releases', 1),
 ('Etexts', 3),
 ('are', 2917),
 ('placed', 10),
 ('certain', 116),
 ('implications', 1),
 ('you', 9081),
 ('should', 1387),
 ('ELECTRONIC', 442),
 ('VERSION', 221),
 ('COMPLETE', 223),
 ('WORKS', 221),
 ('WILLIAM', 244),
 ('SHAKESPEARE', 223),
 ('COPYRIGHT', 221),
 ('BY', 663),
 ('PROVIDED', 222),
 ('PROJECT', 222),
 ('BENEDICTINE', 221),
 ('COLLEGE', 221),
 ('PERMISSION.', 221),
 ('COPIES', 442),
 ('MAY', 223),
 ('DISTRIBUTED', 441),
 ('SO', 224),
 ('LONG', 221),
 ('(1)', 222),
 ('FOR', 669),
 ('OR', 672),
 ('OTHERS', 233),
 ('PERSONAL', 221),
 ('USE', 221),
 ('ONLY,', 221),
 ('(2)', 222),
 ('USED', 221),
 ('COMMERCIALLY.', 221),
 ('DISTRIBUTION', 221),
 ('ANY', 223),
 ('THAT', 222),
 ('CHARGES', 223),
 ('TIME', 221),
 ('*Project', 1),
 ('proud'

In [22]:
pets = sc.parallelize([("cat", 1), ("dog", 1), ("cat", 2), ("dog", 3)])
